In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.pyplot as plt
import xgboost
from datetime import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from tabulate import tabulate
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.linear_model import Perceptron
from timeit import timeit
from sklearn import datasets, tree
import datetime
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
import pylab 
sns.set(style="ticks", color_codes=True, font_scale=1.5)
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
%matplotlib inline
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
#from graphviz import Source
from IPython.display import Image
from scipy.stats import skew, norm, probplot, boxcox, f_oneway
from scipy import interp
from sklearn.base import BaseEstimator, TransformerMixin, clone, ClassifierMixin
from sklearn import metrics, tree
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler, PolynomialFeatures, MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.cluster import DBSCAN
#from mlxtend.classifier import StackingClassifier
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
import scipy
from sklearn.model_selection import GridSearchCV
from google.cloud import bigquery
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import chi2_contingency

In [2]:
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=gBwcRGw0UMtx4f1YlatmLdZm3mjoZK&access_type=offline&code_challenge=0k-Se6ylgOfctAc3GoJV0J84PkvILeSav6dzXUcDQEY&code_challenge_method=S256


Credentials saved to file: [C:\Users\luish\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-default set-quota-project to add a 

In [4]:
query = client.query("""
   SELECT *
   FROM 
       tables_staging.df_models
   WHERE CUSTOMER_ACCOUNT_NR_MASK IN (40119557, 51435241, 59803648, 83372832, 62735475, 71493852,
       3117308, 83765317, 62866214, 83724551, 83395000, 60012661,
       110708204, 83401618, 19243178, 83393777, 19394511, 8736972,
       87415946, 87715856, 58274472, 92959381, 332953, 8615531, 87555786,
       29416697, 72974206, 35329337, 8675376, 85717817, 3475678, 5962226,
       12276650, 26402016, 29488074, 63137804, 59782362, 79515226,
       65720584, 19523710, 69219956, 83563372, 19204079, 87651017,
       84682115, 87408632, 9139989, 87570575, 80239180, 35330137,
       19341334, 83784173, 3065245, 83019383, 58475510, 54246765,
       87589339, 83346120, 35103099, 53499938, 12223508, 13376772,
       12515785, 83465654, 79591075, 9482519, 40171265, 19228548,
       75905769, 83667202, 11729338, 83521436, 34800212, 63234171,
       2789797, 59755436, 8587555, 18911243, 87499587, 58367546, 58113416,
       83408558, 83664384, 18915996, 396634, 26600478, 4280749, 39680217,
       34946074, 12257542, 40226426, 35136756, 50849946, 34937202,
       40053636, 58139794, 18970164, 39776678, 75985833, 59989188, 457300,
       80283247, 59844166, 25900167, 34586106, 19031006, 9059566,
       19342312, 58264850, 1172015, 80378375, 58329048, 80354647,
       83480632, 103388170, 9842261, 34614938, 50836946, 83436203,
       50800943, 59675134, 83594599, 83282531, 83559746, 79640490,
       29453248, 947350, 26364901, 39981995, 83400941, 59729184, 10532324,
       80362603, 58270167, 83462435, 34586076, 11665007, 5486230,
       70799511, 19529623, 83442272, 12120644, 34725020, 40182046,
       3856317, 92839037, 59925221, 8145688, 34880990, 102980508,
       93787422, 60692613, 51327660, 82786308, 87467539, 83700631,
       6645683, 2523041, 13048293, 26460149, 83490935, 87462021, 4252015,
       5055905, 66725820, 3194088, 83515786, 59951871, 40308501, 58526332,
       26269376, 59924568, 66881061, 12167956, 59808687, 5230429,
       29304039, 19017838, 102726295, 59712747, 87471347, 25624350,
       39741603, 7248545, 93170620, 87386240, 58186213, 62768564,
       83685018, 58523802, 35028261, 87614604, 19361406, 7954243,
       71484657, 87649758, 5932760, 59923935, 93586455, 72941082,
       87471817, 2849286, 39592340, 58316652, 34950147, 60007334, 9832521,
       1072151, 77675855, 71561002, 71506017, 34567526, 58360541, 6164386,
       59740423, 100574714, 79638979, 87379555, 50967359, 12557260,
       59699781, 19480086, 82605528, 83530334, 58221053, 9758155,
       80328203, 87479340, 50282366, 39999611, 34869011, 19111769,
       71669267, 4168052, 103122904, 94218235, 83655957, 87494218,
       83526677, 75920841, 80328404, 107612518, 10801631, 59728494,
       34925450, 129693269, 10344850, 87647076, 59806333, 608850,
       34902092, 10553374, 58226070, 51274788, 39733671, 58416269,
       18893240, 130366108, 7774258, 26491740, 25879492, 34610793,
       83771885, 1593868, 9805405, 18974058, 94544323, 66964628, 10105631,
       80360482, 39630514, 6867147, 59965372, 9155436, 79644159, 58277624,
       87422715, 91325472, 103296203, 2980801, 26502467, 8128139,
       63151674, 59852517, 1981739, 89937892, 35268006, 1610551, 83775304,
       102941524, 34921742, 34881653, 26585161, 5079193, 93879272,
       6554495, 59817860, 1916490, 19063947, 59965529, 34618401, 60573496,
       35241534, 35122644, 87610270, 37912, 62836571, 12177802, 19363799,
       83630572, 83486937, 19575511, 84465474, 26400246, 71593733,
       11852324, 29321769, 83428501, 25616425, 93200159, 80239063,
       50698317, 66775915, 1967596, 58353448, 103134579, 19595267,
       59884707, 40023054, 83338190, 58427141, 2945756, 59981970,
       83706532, 40350785, 80194920, 13046181, 99619210, 79579977,
       9574087, 2129104, 67059108, 66979231, 19546916, 83605811, 51480876,
       59854123, 79624842, 83738446, 4092679, 13283749, 2903695, 83611477,
       58149188, 58192769, 83535649, 58441256, 5834654, 35027402,
       34764938, 66625212, 83130588, 92286365, 59839642, 19575323,
       5828081, 19449480, 79634534, 58196114, 76016359, 83753913,
       26442762, 35006354, 60807447, 40022328, 34841266, 18933798,
       39850588, 40169462, 19006325, 83665040, 39756194, 26322109,
       93070334, 102752599, 18949928, 83430599, 25797019, 66871006,
       103133585, 87616802, 66761145, 8607458, 26502565, 59897913,
       29480655, 2887859, 85304894, 80220912, 58505212, 34992565,
       69474696, 29276526, 40232464, 3611317, 78120804, 34939564,
       80202552, 58228524, 80357713, 19286793, 19230069, 29488212,
       71581345, 19083528, 59847659, 101372847, 19586197, 40220965,
       80232412, 34775748, 18949341, 10257405, 19113568, 40097852,
       76415707, 87643671, 59702734, 2944345, 80202170, 39633120,
       35101316, 9544024, 19000317, 19633437, 9529316, 40084053, 59778528,
       12962537, 80249066, 78590280, 88093667, 102693642, 10894040,
       5926514, 19572027, 3467070, 58356243, 3700492, 19582754, 99922113,
       13031606, 67321986, 51136225, 81667387, 731884, 58316009, 471864,
       11315241, 83491620, 87369444, 58302844, 35238063, 81548965,
       58423449, 94365211, 5349257, 59846952, 40348254, 58459782,
       100225690, 35019444, 295501, 75808302, 59902851, 19243813, 9007266,
       83400043, 75747848, 40292044, 19489701, 35300600, 80463017,
       91371247, 7125823, 87618018, 83771436, 9335132, 19143595, 87569905,
       5518229, 87531333, 72574337, 58476258, 83401156, 12980036,
       58516324, 78121793, 19383196, 35142649, 87495170, 83666605,
       1242153, 34813125, 102457504, 87419772, 81656321, 58455302,
       19190685, 83364178, 58375677, 94197152, 26387868, 79547988,
       34811172, 75827661, 59956735, 12871209, 19589390, 6227527, 7193509,
       19185625, 83709440, 10654018, 19305641, 11257237, 2003984,
       83690941, 8063255, 112467482, 79600241, 84760514, 78142110,
       26346002, 26518495, 71524187, 26390959, 86005820, 86203769,
       26475795, 6946384, 60922670, 51078948, 83679888, 19635030, 5070212,
       8406386, 1460413, 1133209, 66695274, 66818932, 69726411, 681704,
       58208210, 77826003, 19210009, 40285783, 29461572, 9493177,
       40232955, 79553779, 58284638, 19437256, 34997877, 104564558,
       3418370, 62874679, 5837307, 58156906, 58495505, 84818244, 34765018,
       3083880, 10894888, 59739148, 12515336, 81622416, 5900481, 34709877,
       100226549, 3796294, 6244357, 58156989, 19317441, 83382861,
       79540099, 59967820, 26314190, 89951970, 83576584, 29337923,
       29347236, 53814005, 13363691, 66633458, 80239597, 83300633,
       34959503, 86652832, 2890028, 84509764, 92115857, 19593919,
       59687794, 34640970, 83382361, 87516024, 83533468, 13171827,
       3759563, 59920428, 18939620, 83792313, 12980061, 34788746,
       128398028, 19274514, 39820803, 1119779, 26618701, 26518659,
       39977619, 59768083, 58508502, 19514535, 87378212, 34983607,
       40254248, 26296755, 87384647, 71501592, 39936339, 87630657,
       40270915, 18910735, 10139080, 58283898, 80171891, 9298060,
       71515022, 83401731, 6589303, 50514112, 80339337, 92194517,
       18885480, 100188611, 55031571, 83365219, 40326822, 83364884,
       58275961, 9303140, 40196046, 8233719, 59891701, 83731716, 26378476,
       59760632, 81590621, 82995451, 26522464, 79565264, 58226234,
       125228790, 50922567, 4081943, 87679585, 19598367, 19394771, 177299,
       87560910, 7152560, 26513514, 4458410, 4210887, 99790629, 26578057,
       11590670, 6359875, 29380088, 19342043, 59670237, 10640319,
       77579570, 2381096, 50522425, 5592639, 76039268, 7943400, 26489265,
       26601946, 26521517, 58277118, 83425959, 93545941, 76019573,
       35255400, 39674683, 79616479, 34641300, 12845306, 83518781,
       34849271, 58158708, 682808, 102809096, 26535189, 66630660,
       39661494, 19211739, 83522448, 58442737, 72956823, 83384197,
       39748303, 58392459, 51101589, 77836267, 7923839, 83380901,
       100545759, 83764509, 120567137, 83456837, 59976140, 59671771,
       61416621, 58496620, 83349149, 39824254, 75691228, 83358751,
       87494794, 13025896, 11627871, 4095466, 83461665, 11172871,
       39994483, 58447894, 63397481, 75964143, 83594729, 80165689,
       26513722, 59986777, 83513598, 94173326, 39860814, 79644549,
       2659406, 102967863, 35005435, 18914457, 82993917, 26569507,
       83578799, 18979272, 26544416, 51349560, 35094713, 83734857,
       1881021, 19338904, 39848592, 60839046, 58367725, 12293354,
       26252886, 26253630, 80175572, 29284158, 26513012, 50636886,
       26563113, 2455268, 2109697, 80323953, 34829163, 58513193, 35140443,
       80186376, 51376374, 3454725, 80356295, 83597079, 79651680,
       83415009, 101281922, 19398541, 29468484, 40108582, 71475037,
       26269953, 83465494, 94050482, 4360222, 87677276, 71558075, 7300774,
       93134054, 58196507, 39823522, 34791838, 423754, 71585779, 2010692,
       87379801, 29365545, 83413734, 39854202, 87494793, 76058386,
       9858884, 85405019, 58112970, 13337486, 107680813, 35302000,
       9354586, 39858422, 10900743, 87611427, 202298, 18917984, 34732848,
       58383372, 62813306, 4820245, 67146581, 2021880, 6347907, 83461166,
       83472149, 13248069, 100196751, 6625496, 87606784, 26305411,
       83704924, 59998127, 26589462, 59973635, 19620636, 3682505,
       26589534, 100184276, 26400080, 40223973, 7117945, 4271022,
       59779125, 51425146, 83738363, 83414622, 26531294, 58344588,
       83721455, 35231900, 35245772, 39736627, 83434713, 1965318,
       75936022, 103454720, 58160608, 66889077, 19337241, 81722298,
       39589733, 25527026, 10587668, 35203400, 50728612, 12504304,
       87629398, 39808384, 7395471, 83794224, 58480922, 84500782,
       58524204, 93176715, 9749751, 9219171, 26518349, 87492114, 39847892,
       29430523, 29468396, 179359, 10911303, 80387642, 11779601, 35232883,
       34900016, 34571275, 66788982, 4688795, 34761632, 66898004,
       87456189, 8579571, 105950796, 58388130, 4509675, 40288817,
       83446050, 79645625, 10925652, 67108875, 1515228, 3590850, 34579811,
       58180839, 83378863, 54706275, 92226870, 58385128, 75770720,
       92297638, 87715510, 50745069, 35132468, 87614759, 58388694,
       59703994, 4502544, 19454531, 26306475, 63351963, 35300245, 5090378,
       94452889, 71229353, 40347802, 83727639, 19081325, 40248047,
       40215452, 13070182, 25819293, 19262813, 9274130, 26495222,
       35185758, 26344568, 40070326, 103497762, 12237973, 7689355,
       59682382, 3472112, 100131756, 19065018, 1051535, 12078622,
       39973308, 9279821, 71543400, 83387196, 87417780, 91437891,
       80704590, 58332519, 66776135, 60017117, 83364844, 29406267,
       69159403, 40304196, 104665451, 11912555, 58407838, 59955036,
       103237070, 9096837, 2231323, 7754367, 71484339, 35032328, 83719657,
       67055680, 29313565, 89428149, 59779535, 18926713, 19441686,
       58157344, 58104487, 103130132, 108243612, 4671866, 19098591,
       51136114, 108200318, 66104531, 26258025, 79628652)
   ORDER BY CUSTOMER_ACCOUNT_NR_MASK ASC, SUBCAT_CD_EXT ASC, YEAR ASC, MONTH ASC
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df # Selected the top1000 customers with the most number of transactions in the last 360 days in 12/2021

,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_MONTH,CUST_NUM_TRANSACTIONS_QUARTER,CUST_NUM_TRANSACTIONS_SEMESTER,CUST_NUM_TRANSACTIONS_YEAR,CUST_TOTAL_QTY_BOUGHT_MONTH,CUST_TOTAL_QTY_BOUGHT_QUARTER,CUST_TOTAL_QTY_BOUGHT_SEMESTER,CUST_TOTAL_QTY_BOUGHT_YEAR,CUST_NUM_UNIQUE_SUBCAT_MONTH,CUST_NUM_UNIQUE_SUBCAT_QUARTER,CUST_NUM_UNIQUE_SUBCAT_SEMESTER,CUST_NUM_UNIQUE_SUBCAT_YEAR,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_BASKET_SIZE_MONTH,CUST_AVG_BASKET_SIZE_QUARTER,CUST_AVG_BASKET_SIZE_SEMESTER,CUST_AVG_BASKET_SIZE_YEAR,SUBCAT_NUM_TRANSACTIONS_MONTH,SUBCAT_NUM_TRANSACTIONS_QUARTER,SUBCAT_NUM_TRANSACTIONS_SEMESTER,SUBCAT_NUM_TRANSACTIONS_YEAR,SUBCAT_TOTAL_QTY_BOUGHT_MONTH,SUBCAT_TOTAL_QTY_BOUGHT_QUARTER,SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,SUBCAT_TOTAL_QTY_BOUGHT_YEAR,SUBCAT_NUM_UNIQUE_CUST_MONTH,SUBCAT_NUM_UNIQUE_CUST_QUARTER,SUBCAT_NUM_UNIQUE_CUST_SEMESTER,SUBCAT_NUM_UNIQUE_CUST_YEAR,CUSTSUBCAT_NUM_TRANSACTIONS_MONTH,CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER,CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER,CUSTSUBCAT_NUM_TRANSACTIONS_YEAR,CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH,CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET
0,37912,10101,1,2021,1,1,5,5,5,5,24,24,24,24,12,12,12,12,2.25,2.25,2.25,2.25,4.20,3.60,3.60,3.60,3.60,2733,2733,2733,2733,4304,4304,4304,4304,1741,1741,1741,1741,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",2,4,101,0
1,37912,10101,2,2021,1,1,6,11,11,11,46,70,70,70,20,27,27,27,4.20,5.30,5.30,5.30,2.40,5.50,4.64,4.64,4.64,2640,5373,5373,5373,4140,8444,8444,8444,1676,2504,2504,2504,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",2,4,101,1
2,37912,10101,3,2021,1,1,11,22,22,22,60,130,130,130,21,37,37,37,2.40,3.86,3.86,3.86,5.20,3.55,4.09,4.09,4.09,2924,8297,8297,8297,4686,13130,13130,13130,1863,3090,3090,3090,2,2,2,2,3,3,3,3,9.00,9.00,9.00,9.00,M,"(3, 8)",2,4,101,1
3,37912,10101,4,2021,2,1,6,6,28,28,29,29,159,159,13,13,39,39,5.20,5.20,4.11,4.11,6.00,4.17,4.17,4.11,4.11,2577,2577,10874,10874,4200,4200,17330,17330,1760,1760,3511,3511,2,2,4,4,4,4,7,7,12.00,12.00,13.00,13.00,M,"(3, 8)",2,4,101,1
4,37912,10101,5,2021,2,1,5,11,33,33,31,60,190,190,13,20,44,44,6.00,5.50,4.38,4.38,3.57,4.20,4.18,4.12,4.12,2824,5401,13698,13698,4815,9015,22145,22145,1885,2731,3890,3890,1,3,5,5,1,5,8,8,0.00,20.50,17.00,17.00,M,"(3, 8)",2,4,101,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365383,130366108,170313,8,2022,3,2,9,21,21,55,67,142,142,463,25,43,43,67,3.00,2.70,2.70,4.28,2.67,6.11,5.19,5.19,5.84,106,191,191,618,190,318,318,912,93,158,158,443,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",1,4,1703,0
3365384,130366108,170313,9,2022,3,2,10,31,31,65,76,218,218,539,25,51,51,69,2.67,2.93,2.93,4.14,4.00,5.80,5.39,5.39,5.83,67,258,258,685,89,407,407,1001,63,207,207,485,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",1,4,1703,0
3365385,130366108,170313,10,2022,4,2,7,7,38,72,18,18,236,557,10,10,54,70,4.00,4.00,3.27,4.20,4.80,1.86,1.86,4.74,5.44,78,78,336,763,118,118,525,1119,71,71,268,533,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",1,4,1703,0
3365386,130366108,170313,11,2022,4,2,6,13,44,78,54,72,290,611,18,24,57,71,4.80,4.33,3.47,4.23,2.75,4.33,3.00,4.68,5.36,77,155,413,840,105,223,630,1224,65,131,316,568,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,M,"(3, 8)",1,4,1703,0


In [5]:
df_v2 = df.copy()

In [7]:
############## DATASET IN DAYS ##############


def replace_zero_last_two_positions_90_days(group):

    rolling_mean = group.rolling(window=3, min_periods=1).mean().astype(int)
    zeros = group == 0  # Boolean index for zero values in the group
    group[zeros] = rolling_mean[zeros]

    return group

def replace_zero_last_two_positions_180_days(group):

    rolling_mean = group.rolling(window=6, min_periods=1).mean().astype(int)
    zeros = group == 0  # Boolean index for zero values in the group
    group[zeros] = rolling_mean[zeros]

    return group

def replace_zero_last_two_positions_360_days(group):

    rolling_mean = group.rolling(window=12, min_periods=1).mean().astype(int)
    zeros = group == 0  # Boolean index for zero values in the group
    group[zeros] = rolling_mean[zeros]

    return group


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS \
                                                    .apply(replace_zero_last_two_positions_90_days) \
                                                    .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS'] \
                                                    .astype(int)


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS \
                                                     .apply(replace_zero_last_two_positions_180_days) \
                                                     .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS'] \
                                                     .astype(int)


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS \
                                                     .apply(replace_zero_last_two_positions_360_days) \
                                                     .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS'] \
                                                     .astype(int)

In [ ]:
df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS \
                                                    .apply(replace_zero_last_two_positions_90_days) \
                                                    .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS'] \
                                                    .astype(int)


df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS \
                                                     .apply(replace_zero_last_two_positions_180_days) \
                                                     .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS'] \
                                                     .astype(int)


df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS \
                                                     .apply(replace_zero_last_two_positions_360_days) \
                                                     .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS'] \
                                                     .astype(int)

In [ ]:
df_v2['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS \
                                                    .apply(replace_zero_last_two_positions_90_days) \
                                                    .reset_index()['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS'] \
                                                    .astype(int)


df_v2['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS \
                                                     .apply(replace_zero_last_two_positions_180_days) \
                                                     .reset_index()['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS'] \
                                                     .astype(int)


df_v2['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT']).CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS \
                                                     .apply(replace_zero_last_two_positions_360_days) \
                                                     .reset_index()['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS'] \
                                                     .astype(int)

In [12]:
#### SAVE DATAFRAME TO BIGQUERY
client.load_table_from_dataframe(df_v2, 'tables_staging.df_models').result()

LoadJob<project=continente-lced-feup, location=europe-southwest1, id=c016ba09-d909-403f-96dd-8621e031204c>

In [7]:
############## FOR DATASET NOT IN DAYS, SO DON'T RUN ##############


def replace_zero_last_two_positions_quarter(group):
    values = group.values
    
    if np.all(values == 0) or np.all(values != 0):
        return group
    
    length = len(group)
    for i in range(-(length - 1), 0):
        if values[i] == 0:
            group.iloc[i] = values[i - 1]

    return group

def replace_zero_last_two_positions_semester(group):
    values = group.values
    
    if np.all(values == 0) or np.all(values != 0):
        return group
    
    length = len(group)
    for i in range(-(length - 1), 0):
        if values[i] == 0:
            group.iloc[i] = values[i - 1]

    return group

def replace_zero_last_two_positions_year(group):
    values = group.values
    
    if np.all(values == 0) or np.all(values != 0):
        return group
    
    length = len(group)
    for i in range(-(length - 1), 0):
        if values[i] == 0:
            group.iloc[i] = values[i - 1]

    return group


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR', 'QUARTER']).CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER \
                                                        .apply(replace_zero_last_two_positions_quarter) \
                                                        .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR','SEMESTER']).CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER \
                                                        .apply(replace_zero_last_two_positions_semester) \
                                                        .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_NUM_TRANSACTIONS_YEAR'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR']).CUSTSUBCAT_NUM_TRANSACTIONS_YEAR \
                                                        .apply(replace_zero_last_two_positions_year) \
                                                        .reset_index()['CUSTSUBCAT_NUM_TRANSACTIONS_YEAR'] \
                                                        .astype(int)

df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR','QUARTER']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER \
                                                        .apply(replace_zero_last_two_positions_quarter) \
                                                        .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR','SEMESTER']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER \
                                                        .apply(replace_zero_last_two_positions_semester) \
                                                        .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR']).CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR \
                                                        .apply(replace_zero_last_two_positions_year) \
                                                        .reset_index()['CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR'] \
                                                        .astype(int)

df_v2['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR','QUARTER']).CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER \
                                                        .apply(replace_zero_last_two_positions_quarter) \
                                                        .reset_index()['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR','SEMESTER']).CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER \
                                                        .apply(replace_zero_last_two_positions_semester) \
                                                        .reset_index()['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER'] \
                                                        .astype(int)


df_v2['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR'] = df_v2.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR']).CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR \
                                                        .apply(replace_zero_last_two_positions_year) \
                                                        .reset_index()['CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR'] \
                                                        .astype(int)